In [1]:
# Make necessary imports
from pyspark import SparkConf
from pyspark.context import SparkContext

In [1]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 64 kB/s  eta 0:00:011
     |████████████████████████████████| 198 kB 3.5 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=bbb9b88e16692f238964bf2722cffcb3a6c5921ad578d7506f6a58bed2db29c6
  Stored in directory: /Users/wanqi/Library/Caches/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Make necessary imports
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.mllib.random import RandomRDDs
from pyspark.sql import SQLContext
from pyspark import sql
import numpy as np

In [2]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
sqlContext = sql.SQLContext(sc) #?

/opt/anaconda3/lib/python3.7/site-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [3]:
def normalization(x):
    s = sum(x[1])
    s_last = 0
    x_new = []
    for i in range(len(x[1])-1):
        x_new.append(x[1][i] / s)
        s_last += x_new[i]
    x_new.append(1 - s_last)
    return (x[0], np.array(x_new))

def create_uniform_rdd_rowindexed_matrix(sc, nrow, ncol, normalize=False):
    rdd = []
    for i in range(nrow):
        rdd.append((i, np.random.rand(ncol)))
    #print(rdd)
    rdd = sc.parallelize(rdd)
    if normalize:
        rdd = rdd.map(normalization)
    return rdd

In [4]:
H = create_uniform_rdd_rowindexed_matrix(sc, 4, 4, normalize=True)
H.collect()

[(0, array([0.00788219, 0.16241676, 0.40886796, 0.42083308])),
 (1, array([0.2991057 , 0.33809772, 0.30567618, 0.05712041])),
 (2, array([0.01124907, 0.57472508, 0.31160588, 0.10241997])),
 (3, array([0.4245387 , 0.3712556 , 0.01716775, 0.18703795]))]

In [15]:
h = H.collectAsMap()

In [16]:
h[0]

array([0.49727713, 0.08730266, 0.14581416, 0.26960605])

In [10]:
n =  sc.broadcast(H.collect())

In [14]:
n.value[0][1]

array([0.27273138, 0.15782045, 0.24609834, 0.32334983])

In [17]:
m = sc.broadcast(H.collectAsMap())

In [20]:
m.value[1]

array([0.288814  , 0.21350555, 0.46121729, 0.03646315])

In [17]:
W = create_uniform_rdd_rowindexed_matrix(sc, 4, 4)
W.collect()

[(0, array([0.49352253, 0.3193459 , 0.7257946 , 0.32108793])),
 (1, array([0.49692689, 0.23458883, 0.44967443, 0.79594515])),
 (2, array([0.05980303, 0.90236792, 0.67384485, 0.33462656])),
 (3, array([0.61481332, 0.77953192, 0.51337314, 0.43117152]))]

In [5]:
def update_h(hxy_tuple):
    h, x, y = hxy_tuple
    d = h * y 
    e = h * x
    #h_new = h * (x+d) / (y+e)
    h_new = h * np.divide(x+d, y+e)
    return h_new

In [6]:
class MRAlgorithm:
    def __init__(self, datapath):
        self.sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
        self.input = self.sc.textFile(datapath).map(lambda x: (int(x.split(',')[0]),
                                                               int(x.split(',')[1]),int(x.split(',')[2])))
        
    def __init__(self, datapath, k):
        self.k = k
        self.n = 2649429
        self.m = 17770
        self.sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
        self.input = self.sc.textFile(datapath).map(lambda x: (int(x.split(',')[0])-1,
                                                               int(x.split(',')[1])-1,int(x.split(',')[2])))
        self.A = self.input
        self.H = create_uniform_rdd_rowindexed_matrix(self.sc, self.n, self.k, normalize=True)
        self.W = create_uniform_rdd_rowindexed_matrix(self.sc, self.m, self.k)
        
    def compute_H(self):
        print("Compute H")
    
        ## compute X
        # M1
        W_b = sc.broadcast(self.W.collect()) # a dictionary where the key is the index i
        Aw = self.A.map(lambda x: (x[1], x[2]*W_b.value[x[0]][1])) #Wi.T
        #print(Aw.collect())
        # R1 
        X = Aw.reduceByKey(lambda x, y: x+y) # add a combiner? #.map(lambda x: (x[0], x[1]))
    
        
        ## compute B
        # M2
        self.ww = self.W.map(lambda x: (None, x[1]@x[1].T)) # w.T.dot(w)? #import numpy.linalg
        # from pyspark.mllib.linalg.distributed import *
        # as_block_matrix(w.T).multiply(as_block_matrix(w))
        # https://stackoverflow.com/questions/37766213/spark-matrix-multiplication-with-python
        # R2
        self.B = self.ww.reduceByKey(lambda x, y: x+y).map(lambda x: x[1]) # without map?
        
        ## compute Y
        # M3
        self.B_b = sc.broadcast(self.B.collect())
        self.y = self.H.map(lambda x: (x[0], self.B_b@x[1]))
        
        ## update H
        # M4
        #self.hxj = self.H.join(self.y).join(self.X)
        # R4
        # why is it a reduce phase instead of a map phase?
        #self.hnew = self.hxj.reduceByKey(lambda x: (x[0], update_h(x[1])))
        #print(self.hnew)
        #self.H = self.hnew

    def compute_W(self):
        print("Compute W")
        #M5: Compute matrix S (repartition join of H and A)
        tmp_M5 = self.H.join(self.A.map(lambda x: (x[1], (x[0], x[2]))))

        #R5: Emit each hj according to row i
        tmp_M5 = tmp_M5.map(lambda x: (x[1][1][0], x[1][1][1] * x[1][0]))

        #M6: Finish the computation of matrix S -- identity mapper so no code needed
        #R6: Emit each row in matrix S
        S = tmp_M5.reduceByKey(lambda x, y : x +y)

        #M7: Calculate matrix C (same as calculating B in compute_H)
        tmp_M7 = self.H.map(lambda x: (0, np.outer(x[1], x[1].T)))
        #R7: Emit matrix C from the sum of matrices
        C = tmp_M7.reduceByKey(lambda x, y : x +y).map(lambda x: x[1])

        #M8: Compute matrix T
        C_b = sc.broadcast(C.collect())
        T = self.W.map(lambda x: (x[0], x[1]@C_b))
        print(T.collect())
        print(S.collect())

        #M9: update W with broadcast join of S and T. W = W * S/T
        T_b = self.sc.broadcast(T.collectAsMap())
        S_b = self.sc.broadcast(S.collectAsMap())
        self.W = self.W.map(lambda x: (x[0], x[1]*np.divide(S_b.value[x[0]], T_b.value[x[0]])))

        print(self.W)

        

        
    def assign_clusters(self):
        #M10: Map user to cluster with highest probability
        self.clusters = self.H.map(lambda x: (x[0], x[1].index(max(x[1]))))
        
        #M11: Emit a 1 for each user that is in a cluster
        self.clustersizes = self.clusters.map(lambda x: (x[1], 1))
        
        #R11: Count the number of users per cluster
        self.clustersizes = self.clustersizes.reduceByKey(lambda x,y: x+y)
        
    def RM2_distribution(self):
        #M12: Emit each rating that a user gave
        self.ratings = self.input.map(lambda x: (x[1], x[2]))
        
        #R12: Sum ratings given by user
        self.ratings = self.ratings.reduceByKey(lambda x, y: x+y)
        
        #Accumulator containing total count of ratings
        globalcount = self.sc.accumulator(0)
        self.ratings.foreach(lambda x: globalcount.add(x[1]))
        
        #M13: Emit each rating that an item received
        self.items = self.input.map(lambda x: (x[0], x[2]))
        
        #Broadcast the total count of ratings, to be used in next reduce
        globalcount = self.sc.broadcast(globalcount.value)
        
        #R13: Compute the probability of item in the collection
        self.items = self.items.reduceByKey(lambda x,y: x+y)
        
        #Divide previous values by the total count of ratings, to normalize probabilities
        self.items = self.items.map(lambda x: (x[0], x[1]/globalcount.value))
        
        #Broadcast all users and which cluster they are in
        clusters = self.sc.broadcast(self.clusters.collectAsMap())
        
        #M14-a: Map each rating from input to the cluster assigned to user
        self.input2cluster = self.input.map(lambda x: (clusters[x[1]], (x[1], x[0], x[2])))
        
        #M14-b: Map sum of ratings of a user to their assigned cluster
        self.ratings2cluster = self.ratings.map(lambda x: (clusters[x[0]], (x[0], x[1])))
        
        #Repartition join these two maps by cluster
        self.fullClusters = self.input2cluster.join(self.ratings2cluster)
        
        #Broadcast sizes of the clusters
        clustersizes = self.sc.broadcast(self.clustersizes.collectAsMap())
        
        #Broadcast probabilities of each item
        items = self.sc.broadcast(self.items.collectAsMap())
        
        #Final reduce combines all information from above, needs W and H

In [7]:
Algorithm = MRAlgorithm('../data/small_dataset.txt', k=5)
Algorithm.compute_H()
Algorithm.assign_clusters()
Algorithm.RM2_distribution()

Compute H


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 6.0 failed 1 times, most recent failure: Lost task 5.0 in stage 6.0 (TID 41) (129.104.246.210 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/opt/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 611, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 74, in wrapper
    return f(*a

In [51]:
Algorithm.input.take(10)

[(1, 1488844, 3),
 (1, 822109, 5),
 (1, 885013, 4),
 (1, 30878, 4),
 (1, 823519, 3),
 (1, 893988, 3),
 (1, 124105, 4),
 (1, 1248029, 3),
 (1, 1842128, 4),
 (1, 2238063, 3)]

In [52]:
Algorithm.clusters.collectAsMap()

{1: 2, 2: 1, 3: 0, 4: 0}

In [53]:
Algorithm.clustersizes.collect()

[(0, 2), (1, 1), (2, 1)]

In [54]:
Algorithm.ratings.take(5)

[(1488844, 9), (30878, 8), (893988, 3), (1842128, 4), (2207774, 5)]

In [55]:
Algorithm.items.take(5)

[(2, 0.0031263442977540004),
 (4, 0.002356875836872686),
 (6, 0.019042829705117875),
 (8, 0.2881568503898842),
 (10, 0.004798574968645675)]